In [13]:
import numpy as np
import torch
import numba as nb
import torchquad
from scipy import optimize, integrate, linalg
from classes import Region, Coordinate, Demands_generator, Demand


tol = 1e-4
# Instrumental functions for scipy integration
@nb.jit(nopython=True)
def norm_func(x, y):
    return np.sqrt(np.sum(np.square(x - y)))

@nb.jit(nopython=True)
def modified_norm(x_cdnt, i, demands_locations, lambdas):
    return norm_func(x_cdnt, demands_locations[i]) - lambdas[i]

@nb.jit(nopython=True)
def region_indicator(i, x_cdnt, lambdas, demands_locations):
    i_modified_norm = modified_norm(x_cdnt, i, demands_locations, lambdas)
    for j in range(len(lambdas)):
        if i_modified_norm > modified_norm(x_cdnt, j, demands_locations, lambdas):
            return 0
    return 1

@nb.jit(nopython=True)
def categorize_x(x_cdnt, demands_locations, lambdas, v):
    modified_norms = np.array([modified_norm(x_cdnt, i, demands_locations, lambdas) for i in range(demands_locations.shape[0])])
    i = np.argmin(modified_norms)
    return demands_locations[i], v[i+1]

# Integrands of function and derivatives
# @nb.jit(nopython=True)
def integrand(X, lambdas, v, demands_locations):
    '''
    X is a n-by-2 matrix, the first column is r, the second column is theta.
    '''
    dtype = X.dtype
    print(f"lambdas is {lambdas}., v0 is {v[0]}, v1 is {v[1:]}.")
    lambdas, v1, v0 = torch.tensor(lambdas, dtype=dtype), torch.tensor(v[1:], dtype=dtype), v[0]
    # print(f"lambdas is {lambdas}., v0 is {v0}, v1 is {v1}.")
    demands_locations = torch.tensor(demands_locations, dtype=dtype)
    x_cdnt = X.clone()
    x_cdnt[:, 0] = X[:, 0]*torch.cos(x_cdnt[:, 1])
    x_cdnt[:, 1] = X[:, 0]*torch.sin(x_cdnt[:, 1])
    norms = torch.cdist(x_cdnt, demands_locations, p=2)
    modified_norms, modified_norms_indices = torch.min(norms - lambdas, dim=1)
    raw_intgrd = 1 / (v0*norms[torch.arange(norms.shape[0]), modified_norms_indices] + v1[modified_norms_indices])
    if raw_intgrd.is_complex(): 
        # print(f"raw_intgrd is complex. {raw_intgrd}")
        raise ValueError
    return X[:, 0]*raw_intgrd

@nb.njit
def integrand_scipy(r, theta, lambdas, v, demands_locations):
    x_cdnt = np.array([r*np.cos(theta), r*np.sin(theta)])
    xi, vi = categorize_x(x_cdnt, demands_locations, lambdas, v)
    raw_intgrd = 1 / (v[0]*norm_func(x_cdnt, xi) + vi)
    return r*raw_intgrd

@nb.njit
def jac_integrand0(r, theta, lambdas, v, demands_locations):
    x_cdnt = np.array([r*np.cos(theta), r*np.sin(theta)])
    xi, vi = categorize_x(x_cdnt, demands_locations, lambdas, v)
    the_norm = norm_func(x_cdnt, xi)
    return -r * the_norm/pow(v[0]*the_norm + vi, 2)

'''NOTE: i is the index of v, whose values range from 0 to n
      But j is the index of regions, whose values range from 1 to n.
      For indexes of demands_locations (0 to n-1), we need to offset j by subtracting one.'''

@nb.njit
def jac_integrandj(r, theta, lambdas, v, demands_locations, j):
    x_cdnt = np.array([r*np.cos(theta), r*np.sin(theta)])
    if region_indicator(j-1, x_cdnt, lambdas, demands_locations) == 0: return 0
    return -r / pow(v[0] * norm_func(x_cdnt, demands_locations[j-1]) + v[j], 2)

def objective_function(v, demands_locations, lambdas, t, region_radius, thetarange):
    # print(f"DEBUG: v is {v, type(v[1])}.")
    # v = np.real(v)
    start, end = thetarange
    simpson = torchquad.Simpson()
    sum_integral = simpson.integrate(lambda X: integrand(X, lambdas, v, demands_locations), dim=2, N=10000000, integration_domain=[[0, region_radius], [start, end]], backend='torch').item()
    # print(f"sum_integral is {sum_integral}. with type {type(sum_integral)}")
    return 1/4*sum_integral + v[0]*t + np.mean(v[1:])

def objective_function_scipy(v, demands_locations, lambdas, t, region_radius, thetarange):
    print(f"DEBUG: v is {v, type(v[1])}.")
    start, end = thetarange
    sum_integral, error = integrate.dblquad(integrand_scipy, start, end, lambda _: 0, lambda _: region_radius, args=(lambdas, v, demands_locations), epsabs=tol)
    return 1/4*sum_integral + v[0]*t + np.mean(v[1:])


def objective_jac(v: list[float], demands_locations: list[list[float]], lambdas: list[float], t: float, region_radius, thetarange):
    start, end = thetarange
    n = demands_locations.shape[0]
    jac = np.zeros(n + 1)
    jac[0] = 1/4 * integrate.dblquad(jac_integrand0, start, end, lambda _: 0.00001, lambda _: region_radius, args=(lambdas, v, demands_locations), epsabs=tol)[0] + t
    for j in range(1, n+1):
        jac[j] = 1/4 * integrate.dblquad(jac_integrandj, start, end, lambda _: 0.00001, lambda _: region_radius, args=(lambdas, v, demands_locations, j), epsabs=tol)[0] + 1/n
    print(f"DEBUG: jac is {jac}.")
    return jac

def minimize_problem7(lambdas, demands, thetarange, t, region_radius):
    # constraints = [optimize.NonlinearConstraint(lambda v: constraint_func(lambdas, demands, v, region_radius), 0, np.inf)]
    demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
    bounds = optimize.Bounds(0, np.inf)
    result = optimize.minimize(objective_function, x0=np.append(1e-6, np.ones(demands.shape[0])), args=(demands_locations, lambdas, t, region_radius, thetarange), jac=objective_jac, method='SLSQP',  bounds=bounds, options={'ftol': tol, 'disp': True})
    return result.x, result.fun

region = Region(1)
depot = Coordinate(2, 0.3)
# generator = Demands_generator(region, 10)
t, epsilon = 0.25, 0.1

thetarange = (3.147444264116321 - 2*np.pi, 0.00012460881117814946)
demands = np.array([Demand(Coordinate(r = 0.1802696888767692, theta = 4.768809396779301), 1),
           Demand(Coordinate(r = 0.019475241487624584, theta = 5.1413757057591045), 1),
           Demand(Coordinate(r = 0.012780814590608647, theta = 4.478189127165961), 1),
           Demand(Coordinate(r = 0.4873716073198716, theta = 3.7670422583826495), 1),
           Demand(Coordinate(r = 0.10873607186897494, theta = 5.328009178184025), 1),
           Demand(Coordinate(r = 0.8939041702850812, theta = 4.5103794163992506), 1),
           Demand(Coordinate(r = 0.8571542470728296, theta = 3.7828800007876318), 1),
           Demand(Coordinate(r = 0.16508661759522714, theta = 3.4707299112070515), 1),
           Demand(Coordinate(r = 0.6323340138234961, theta = 5.963386240530588), 1),
           Demand(Coordinate(r = 0.020483612791232675, theta = 6.19945137229428), 1),
           Demand(Coordinate(r = 0.15791230667493694, theta = 5.004153427569559), 1)])
demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
lambdas = np.zeros(demands.shape[0])

minimize_problem7(lambdas, demands, thetarange, t, region.radius)
# objective_function(np.append(1e-6, np.ones(demands.shape[0])), demands_locations, lambdas, t, region.radius, thetarange), objective_function_scipy(np.append(1e-6, np.ones(demands.shape[0])), demands_locations, lambdas, t, region.radius, thetarange)

lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 1e-06, v1 is [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.].


/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/scipy/integrate/_quadpack_py.py:1233: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/scipy/integrate/_quadpack_py.py:1233: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/simpson.py:77: UserWarning: N 

DEBUG: jac is [ 0.15237174  0.05542311  0.09009643  0.08939329  0.03347948  0.07921376
  0.00754656  0.02935289  0.07738335 -0.02124277  0.09025623  0.07757027].
lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 0.0, v1 is [0.94457689 0.90990357 0.91060671 0.96652052 0.92078624 0.99245344
 0.97064711 0.92261665 1.02124277 0.90974377 0.92242973].


/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/utils.py:255: UserWarning: DEPRECATION WARNING: In future versions of torchquad, an array-like object will be returned.
  warnings.warn(


DEBUG: jac is [ 0.14951365  0.05113667  0.0899275   0.08908394  0.02943182  0.07711606
  0.00627393  0.02557359  0.07501903 -0.01662556  0.09012019  0.07523243].
lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 0.0, v1 is [0.68750527 0.45800925 0.46294783 0.81852282 0.53322173 0.96089473
 0.8420439  0.54558317 1.10490268 0.45688203 0.54432456].
DEBUG: jac is [ 0.11997265  0.01583207  0.08704035  0.08383255  0.00519107  0.04977769
  0.00062514  0.00409261  0.04546735 -0.00095815  0.08778183  0.04588935].
lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 0.0, v1 is [0.44244058 0.         0.         0.68535058 0.11137665 0.93346447
 0.72716477 0.14118426 1.17622898 0.         0.13818395].
lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 0.0, v1 is [0.6629988  0.41220832 0.41665305 0.80520559 0.49103722 0.9581517
 0.83055599 0.50514327 1.11203531 0.41119383 0.5037105 ].
DEBUG: jac is [1.13988272e-01 1.01796139e-02 8.61325530e-02 8.21728853e-02
 2.33174192e-03 4.24079353e-02 1

(array([3.46391326e-18, 6.14822318e-01, 9.65945463e-02, 1.34070553e-01,
        7.83603013e-01, 3.58985231e-01, 9.53966571e-01, 8.12286499e-01,
        3.85096938e-01, 1.12337054e+00, 8.45804876e-02, 3.82777536e-01]),
 1.046061192010818)

In [14]:
v = 1e-6*np.ones(demands.shape[0]+1)
1/4 * integrate.dblquad(jac_integrand0, start, end, lambda _: 0.00001, lambda _: region.radius, args=(lambdas, v, demands_locations), epsabs=tol)[0] + t

/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/scipy/integrate/_quadpack_py.py:1233: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


-57671770120.76153

In [10]:
start, end = thetarange
start, end = start - 2*np.pi, end
start, end

(-3.1357410430632653, 0.00012460881117814946)

In [4]:
objective_jac(np.append(1e-6, np.ones(demands.shape[0])), demands_locations, lambdas, t, region.radius, thetarange)

DEBUG: jac is [0.47811021 0.09090909 0.09090909 0.09666327 0.12253781 0.09432201
 0.09090909 0.09437652 0.19855623 0.17050254 0.2527685  0.09090909].


array([0.47811021, 0.09090909, 0.09090909, 0.09666327, 0.12253781,
       0.09432201, 0.09090909, 0.09437652, 0.19855623, 0.17050254,
       0.2527685 , 0.09090909])

In [24]:
gaussian = torchquad.Gaussian()
def integrand(X):
    return torch.ones(X.shape[0])
gaussian.integrate(integrand, dim=1, N=1000000, integration_domain=[[0, 1]], backend='torch').item()

In [9]:
-1/4 % 2

1.75